In [1]:
from jenga.tasks.openml import OpenMLBinaryClassificationTask, OpenMLMultiClassClassificationTask, OpenMLRegressionTask
    
import pandas as pd
import numpy as np

from data_imputation_paper.imputation.simple import ModeImputer
from data_imputation_paper.imputation.ml import KNNImputer, ForestImputer
from data_imputation_paper.imputation.dl import AutoKerasImputer
from data_imputation_paper.imputation.generative import GAINImputer, VAEImputer
from data_imputation_paper.evaluation import SingleColumnEvaluator, MultipleColumnsEvaluator, SingleColumnAllMissingEvaluator, MultipleColumnsAllMissingEvaluator

## Make Deterministic

In [2]:
seed = 42

## Create example tasks

In [3]:
task = OpenMLMultiClassClassificationTask(openml_id=4552, seed=seed)

## Insert missing values using jenga

In [4]:
target_columns = ['V2', 'V4', 'V8', 'V15']

## Mode Imputation

In [5]:
arguments = {
    "seed": seed
}

%time SingleColumnEvaluator(task, 0.5, "MCAR", target_columns[0], ModeImputer, arguments).evaluate(3).report()

Evaluation result contains 1 target columns: V2
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2 - Necessary train time in seconds: 0.0011
            train         test
MAE     29.176482    31.314079
MSE   1348.083534  1532.037677
RMSE    36.715550    39.130423

             baseline  corrupted   imputed
F1_micro     0.766108   0.766990  0.766990
F1_macro     0.435556   0.431702  0.431702
F1_weighted  0.751521   0.752002  0.752002


CPU times: user 1.75 s, sys: 154 ms, total: 1.9 s
Wall time: 3.96 s


In [6]:
arguments = {
    "seed": seed
}

%time MultipleColumnsEvaluator(task, 0.5, "MCAR", target_columns[:2], ModeImputer, arguments).evaluate(3).report()

Evaluation result contains 2 target columns: V2, V4
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2 - Necessary train time in seconds: 0.001
            train         test
MAE     29.259324    31.752530
MSE   1347.056370  1585.729435
RMSE    36.698875    39.775974

             baseline  corrupted   imputed
F1_micro     0.766108   0.756693  0.756693
F1_macro     0.435556   0.418234  0.418234
F1_weighted  0.751521   0.740991  0.740991


Target Column: V4 - Necessary train time in seconds: 0.0012
                train      test
F1_micro     0.835834  0.816254
F1_macro     0.455277  0.449400
F1_weighted  0.761114  0.733709

             baseline  corrupted   imputed
F1_micro     0.766108   0.756105  0.756105
F1_macro     0.435556   0.413858  0.413858
F1_weighted  0.751521   0.741196  0.741196


CPU times: user 850 ms, sys: 22.5 ms, total: 873 ms
Wall time: 876 ms


In [7]:
arguments = {
    "seed": seed
}

%time SingleColumnAllMissingEvaluator(task, 0.5, "MCAR", target_columns[-1], ModeImputer, arguments).evaluate(3).report()

Evaluation result contains 1 target columns: V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V15 - Necessary train time in seconds: 0.0012
                train      test
F1_micro     0.134752  0.104762
F1_macro     0.017068  0.016439
F1_weighted  0.032142  0.021778

             baseline  corrupted   imputed
F1_micro     0.766108   0.744925  0.744925
F1_macro     0.435556   0.408127  0.408127
F1_weighted  0.751521   0.729662  0.729662


CPU times: user 549 ms, sys: 14 ms, total: 563 ms
Wall time: 567 ms


In [8]:
arguments = {
    "seed": seed
}

%time MultipleColumnsAllMissingEvaluator(task, 0.5, "MCAR", target_columns[2:], ModeImputer, arguments).evaluate(3).report()

Evaluation result contains 2 target columns: V8, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V8 - Necessary train time in seconds: 0.001
                train      test
F1_micro     0.832151  0.742857
F1_macro     0.454142  0.426127
F1_weighted  0.756018  0.633461

             baseline  corrupted   imputed
F1_micro     0.766108   0.744925  0.744925
F1_macro     0.435556   0.408127  0.408127
F1_weighted  0.751521   0.729662  0.729662


Target Column: V15 - Necessary train time in seconds: 0.001
                train      test
F1_micro     0.134752  0.123810
F1_macro     0.016936  0.020247
F1_weighted  0.032595  0.027534

             baseline  corrupted   imputed
F1_micro     0.766108   0.746102  0.746102
F1_macro     0.435556   0.418958  0.418958
F1_weighted  0.751521   0.731799  0.731799


CPU times: user 891 ms, sys: 14.7 ms, total: 906 ms
Wall time: 908 ms


## KNN imputation

In [9]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_neighbors": [3, 5]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_neighbors": [3, 5]
    }
}

%time MultipleColumnsAllMissingEvaluator(task, 0.5, "MCAR", target_columns, KNNImputer, arguments).evaluate(3).report()

Evaluation result contains 4 target columns: V2, V4, V8, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2 - Necessary train time in seconds: 0.6741
            train         test
MAE     23.630733    30.796190
MSE   1021.804350  1626.473905
RMSE    31.961477    39.941772

             baseline  corrupted   imputed
F1_micro     0.766108   0.744925  0.744337
F1_macro     0.435556   0.408127  0.407696
F1_weighted  0.751521   0.729662  0.729104


Target Column: V4 - Necessary train time in seconds: 0.6836
                train      test
F1_micro     0.962175  0.876190
F1_macro     0.912349  0.805385
F1_weighted  0.960779  0.874140

             baseline  corrupted   imputed
F1_micro     0.766108   0.746102  0.747573
F1_macro     0.435556   0.418958  0.421161
F1_weighted  0.751521   0.731799  0.733299


Target Column: V8 - Necessary train time in seconds: 0.752
                train      test
F1_micro     0.952719  0.885714
F1_macro     0.9322

## Forest imputation

In [10]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_estimators": [50, 100]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_estimators": [50, 100]
    }
}

%time MultipleColumnsAllMissingEvaluator(task, 0.5, "MCAR", target_columns, ForestImputer, arguments).evaluate(3).report()

Evaluation result contains 4 target columns: V2, V4, V8, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2 - Necessary train time in seconds: 4.5031
           train         test
MAE    16.115421    34.405726
MSE   576.005031  1918.184481
RMSE   23.986145    43.572695

             baseline  corrupted   imputed
F1_micro     0.766108   0.747573  0.747279
F1_macro     0.435556   0.411164  0.411002
F1_weighted  0.751521   0.732179  0.731852


Target Column: V4 - Necessary train time in seconds: 1.0595
                train      test
F1_micro     0.981087  0.952381
F1_macro     0.969053  0.927487
F1_weighted  0.981280  0.952936

             baseline  corrupted   imputed
F1_micro     0.766108   0.744925  0.745808
F1_macro     0.435556   0.417928  0.420874
F1_weighted  0.751521   0.730053  0.731135


Target Column: V8 - Necessary train time in seconds: 1.0739
                train      test
F1_micro     0.985816  0.904762
F1_macro     0.980346 

## AutoKeras imputation

In [11]:
arguments = {
    "seed": seed,
    'max_trials': 2,
    'tuner': 'greedy',
    'validation_split': 0.1,
    'epochs': 2
}

%time MultipleColumnsAllMissingEvaluator(task, 0.5, "MCAR", target_columns, AutoKerasImputer, arguments).evaluate(3).report()

Trial 2 Complete [00h 00m 02s]
val_accuracy: 0.10972850769758224

Best val_accuracy So Far: 0.12217194586992264
Total elapsed time: 00h 00m 04s
Epoch 1/2
142/142 [==============================] - 0s 989us/step - loss: 16.3794 - accuracy: 0.0737
Epoch 2/2
142/142 [==============================] - 0s 991us/step - loss: 3.9249 - accuracy: 0.1010


[2021-03-05 17:31:39,965 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6c4fd9d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:31:39,965 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6c4fd9d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:31:39,971 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6974b8b0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:31:39,971 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6974b8b0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


...

[2021-03-05 17:31:40,904 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6c2b48b0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:31:40,910 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6c29c310> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:31:40,910 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6c29c310> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:31:40,916 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6c2a9ca0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:31:40,916 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6c2a9ca0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Evaluation result contains 4 target columns: V2, V4, V8, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2 - Necessary train time in seconds: 9.6596
       

## GAIN imputation

In [12]:
arguments = {
    "seed": seed,
    "hyperparameter_grid": {
        "gain": {
            "alpha": [80, 120],
            "hint_rate": [0.5, 0.9],
            "noise": [0.001, 0.1]
        },
        "training": {
            "batch_size": [64, 256],
            "epochs": [5, 15]
        }
    }
}

%time MultipleColumnsAllMissingEvaluator(task, 0.5, "MCAR", target_columns, GAINImputer, arguments).evaluate(3).report()

[W 2021-03-05 17:34:54,173] Trial 0 failed, because the objective function returned nan.
[W 2021-03-05 17:38:20,426] Trial 8 failed, because the objective function returned nan.
[W 2021-03-05 17:38:50,052] Trial 23 failed, because the objective function returned nan.
[W 2021-03-05 17:39:19,754] Trial 2 failed, because the objective function returned nan.
[W 2021-03-05 17:39:30,695] Trial 8 failed, because the objective function returned nan.
[W 2021-03-05 17:41:04,239] Trial 25 failed, because the objective function returned nan.


Evaluation result contains 4 target columns: V2, V4, V8, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2 - Necessary train time in seconds: 62.9798
            train         test
MAE     33.632874    33.063541
MSE   1923.468327  2086.350232
RMSE    43.617263    45.649971

             baseline  corrupted   imputed
F1_micro     0.766108   0.746396  0.746396
F1_macro     0.435556   0.415422  0.413598
F1_weighted  0.751521   0.731019  0.731112


Target Column: V4 - Necessary train time in seconds: 63.8641
                train      test
F1_micro     0.836879  0.800000
F1_macro     0.483729  0.509691
F1_weighted  0.769120  0.719226

             baseline  corrupted   imputed
F1_micro     0.766108   0.745513  0.745513
F1_macro     0.435556   0.416966  0.416966
F1_weighted  0.751521   0.729771  0.729771


Target Column: V8 - Necessary train time in seconds: 65.1132
                train      test
F1_micro     0.813239  0.857143
F1_macro     0.

## VAE imputation

In [13]:
arguments = {
    "seed": seed,
    "hyperparameter_grid": {
        "training": {
            "batch_size": [64, 256],
            "epochs": [5, 15]
        }
    }
}

%time MultipleColumnsAllMissingEvaluator(task, 0.5, "MCAR", target_columns, VAEImputer, arguments).evaluate(3).report()

[2021-03-05 17:44:39,839 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6a8df0d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:44:39,839 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6a8df0d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:44:39,955 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6a8df0d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:44:39,955 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6a8df0d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:44:50,920 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6a93aaf0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


...

[2021-03-05 17:46:36,088 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e6a8d39d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:46:47,790 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e68493f70> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:46:47,790 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e68493f70> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:46:47,934 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e68493f70> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-05 17:46:47,934 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f9e68493f70> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Evaluation result contains 4 target columns: V2, V4, V8, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2 - Necessary train time in seconds: 10.9681
      